# Agentic AI

## Prerequisites

### Install a Local LLM with Ollama

To run this project locally, we will install and use **Ollama**, a lightweight runtime for local large language models.

**Download Ollama:**  
https://ollama.com/

Once installed, you can pull any model you want to run.  
Below are a few recommended examples, but you are free to pick any size or model from the Ollama library.

ollama pull qwen3:0.6b

or

ollama pull ibm/granite4:350m

or

Choose any model you prefer, make sure the model supports tools.
Browse available models here:
https://ollama.com/library



### Python requirements

In [ ]:
!pip install langgraph langchain-google-genai langchain-core mcp langchain-ollama

## 1. Define FastMCP Tools

In [2]:
from mcp.server.fastmcp import FastMCP
import math

# Initialize FastMCP
mcp = FastMCP("Unified Solver")

@mcp.tool()
def calculate_sum(a: float, b: float) -> float:
    """Calculates the sum of two numbers."""
    return a + b

@mcp.tool()
def calculate_power(base: float, exponent: float) -> float:
    """Calculates the power of a base number."""
    return math.pow(base, exponent)

# TO DO: Add more tools as needed for your application





## 2. LLM + MCP

### 2.1. Global instance of our LLM

In [ ]:
import os
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI


# Choose your model here, can be Ollama or Google Gemini
# Can also switch between different model sizes as needed
model = "qwen3:0.6b"
model = "ibm/granite4:350m"
global_llm = ChatOllama(model=model, temperature=0.0)

# SETUP API KEY if using Google Gemini
os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY_HERE"

# model = "gemini-2.5-flash"
# model = "gemini-2.5-flash-lite"
# global_llm = ChatGoogleGenerativeAI(model=model, temperature=0)


### 2.2. Our agent graph

In [4]:
from langgraph.graph import MessagesState, START, StateGraph
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver # Optional: For saving graph state


def create_agent_graph(sys_msg, tools):
    """ Creates a LangGraph StateGraph with the given tools integrated."""

    llm = global_llm

    if tools:
        llm_with_tools = llm.bind_tools(tools)
    else:
        llm_with_tools = llm

    # Node
    def assistant(state: MessagesState):
        return {
            "messages": [
                llm_with_tools.invoke([sys_msg] + state["messages"], think=False)
            ]
        }

    # Graph
    builder = StateGraph(MessagesState)

    # Define the basic graph structure
    builder.add_node("assistant", assistant)
    builder.add_edge(START, "assistant")

    if tools:
        builder.add_node("tools", ToolNode(tools))  
        builder.add_conditional_edges(
            "assistant",
            tools_condition,
        )
        builder.add_edge("tools", "assistant")

    react_graph = builder.compile()

    return react_graph


async def run_agent(prompt, tools, sys_msg=""):

    sys_msg = SystemMessage(content=sys_msg)

    # 3. Create Graph
    graph = create_agent_graph(sys_msg, tools)
    
    # 4. Run (using ainvoke for async tools)
    config = {"configurable": {"thread_id": "1"}}
    result = await graph.ainvoke({"messages": [HumanMessage(content=prompt)]}, config)

    last_msg = result["messages"][-1].content

    # Extract tool names and outputs
    tools_used = []
    tools_output = []
    
    # Parsing logic specific to your request
    for msg in result["messages"]:
        # In LangChain, tool calls are usually in 'tool_calls' attribute of AIMessage
        # or 'name' attribute if it is a ToolMessage
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
             for tool_call in msg.tool_calls:
                tools_used.append(tool_call['name'])
        
        if msg.type == 'tool':
            tools_output.append(msg.content)

    return last_msg, tools_used, tools_output

### 2.3. Tools that run spacific agent (with tools and without)

In [5]:

@mcp.tool()
async def ask_agent_with_tools(prompt) -> str:
    """ Runs the agent with access to tools."""
    tools = [calculate_sum, calculate_power]
    results = await run_agent(prompt, tools)
    return results[0]

@mcp.tool()
async def ask_agent_without_tools(prompt) -> str:
    """ Runs the agent without access to tools."""
    # return await run_agent(prompt, [])[0]
    results = await run_agent(prompt, [])
    return results[0]


## 3. Run the Test

In [8]:
# THE JUDGE AGENT RUNNER

sys_msg = """
    You are a Research Supervisor. You have two assistants:
    1. 'ask_agent_with_tools': A precise mathematician with a calculator.
    2. 'ask_agent_without_tools': A theorist who estimates mentally.

    Your Goal:
    When a user asks a math question, you must:
    First run ask_agent_with_tools for the answer.
    Second run ask_agent_without_tools for the answer.
    Third Compare their answers in your final response and explain any discrepancies.
    """

prompt = "Compare the two assistants for the question 'What is 5 to the power of 3?'"

tool_list = [ask_agent_with_tools, ask_agent_without_tools]

response, tools, outputs = await run_agent(prompt, tool_list, sys_msg)
print(f"Response: {response}")
print(f"Tools Used: {tools}")
print(f"Tool Outputs: {outputs}")


HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Response: Both assistants agree that 5 raised to the power of 3 equals 125. The first assistant used a calculator, while the second assistant calculated it mentally. Both methods are equivalent for this calculation.
Tools Used: ['ask_agent_with_tools', 'ask_agent_without_tools']
Tool Outputs: ['The result of 5 raised to the power of 3 is 125.', 'The result of 5 raised to the power of 3 (5^3) is 125.']
